In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm

# -----------------------------
# Paths
# -----------------------------
input_dir = "/kaggle/input/augmented_dataset"  # Original or augmented images
output_dir = "/kaggle/working/preprocessed_basic"
classes = ['polyps', 'esophagitis', 'ulcerative-colitis']
IMG_SIZE = (224, 224)

# -----------------------------
# Create output folders
# -----------------------------
for cls in classes:
    os.makedirs(os.path.join(output_dir, cls), exist_ok=True)

# -----------------------------
# Preprocessing: Resize + Normalize + Standardize
# -----------------------------
for cls in classes:
    class_input_path = os.path.join(input_dir, cls)
    class_output_path = os.path.join(output_dir, cls)
    
    image_files = [f for f in os.listdir(class_input_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    
    for img_file in tqdm(image_files, desc=f"Processing {cls}"):
        img_path = os.path.join(class_input_path, img_file)
        img = cv2.imread(img_path)
        if img is None:
            continue
        
        # Resize
        img = cv2.resize(img, IMG_SIZE)
        
        # Normalize [0,1]
        img = img.astype(np.float32) / 255.0
        
        # Standardize (zero mean, unit variance)
        mean = np.mean(img, axis=(0,1), keepdims=True)
        std = np.std(img, axis=(0,1), keepdims=True) + 1e-7
        img = (img - mean) / std
        
        # Convert back to 0-255 for saving
        img_save = ((img - img.min()) / (img.max() - img.min()) * 255).astype(np.uint8)
        
        save_path = os.path.join(class_output_path, img_file)
        cv2.imwrite(save_path, img_save)
